# Chasing Aggregates

It looks like all of our `cml_` fields are hovering around `50%`. That makes is feel like we are calculating the for and against at the same time. This is wrong for all of these fields. 

## Steps to Investigate
- Set up test to `calculate aggregates` for a single year
- Print out some aggregate for that year, week by week for a team
  as well as the other meaningful data for that team

In [1]:
import utils.game_utils as gu
import utils.plot as guplot

import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from utils.data_boss import DataBoss
dboss = DataBoss()


nfld = gu.NFL_Data()

YEAR = 2018
WEEK = 14
TEAM = gu.TEAM_NAME['Saints']

## Calculate Aggregates

In [2]:
# TBD - once we have the data presenting below

## Print Aggregates for the week

Intent here is to see the data for a team by-week so that we can see the aggregation at work. A visualization may be helpful as well.

In [3]:
FIELDS = gu.COMMON_FIELDS + [
  'team_cml_pass_yards_before',
  'team_pass_yards',
  'team_cml_pass_yards_after',
]

teams_df = dboss.teams_df
year_df = gu.get_year(teams_df, YEAR)
team_df = year_df[year_df['team'] == TEAM]
team_df[FIELDS]

,date,year,week,team,team_score,opponent,opponent_score,win,home,team_cml_pass_yards_before,team_pass_yards,team_cml_pass_yards_after
4106,2018-09-09,2018,1,New Orleans Saints,40,Tampa Bay Buccaneers,48,0,1,0,439,439
4144,2018-09-16,2018,2,New Orleans Saints,21,Cleveland Browns,18,1,1,439,243,682
4250,2018-10-08,2018,5,New Orleans Saints,43,Washington Redskins,19,1,1,1305,363,1668
4358,2018-11-04,2018,9,New Orleans Saints,45,Los Angeles Rams,35,1,1,2044,346,2390
4412,2018-11-18,2018,11,New Orleans Saints,48,Philadelphia Eagles,7,1,1,2655,373,3028
4422,2018-11-22,2018,12,New Orleans Saints,31,Atlanta Falcons,17,1,1,3028,171,3199
4570,2018-12-23,2018,16,New Orleans Saints,31,Pittsburgh Steelers,28,1,1,3730,326,4056
4586,2018-12-30,2018,17,New Orleans Saints,14,Carolina Panthers,33,0,1,4056,118,4174


### Checking if accumulation is not working

The below goes through all `cml` fields to see if their `_after_` is equal to `_before + current`. At the time of this writing there were no examples where this was incorrect.

In [4]:
teams_df = dboss.teams_df
year_df = gu.get_year(teams_df, YEAR)

cols = teams_df.columns
team_cml_cols = pd.Series(cols[cols.str.contains('team_cml_')]).sort_values()
cml_cols = team_cml_cols.apply(lambda x: x.replace('team_', '')).values
# cml_cols = team_cml_cols.values

df = teams_df
for col in cml_cols:
  if "_perf" not in col:
    root_field = cml_root_field = col.replace('cml_', '').replace('_before', '').replace('_after', '')
    if root_field == 'points': 
      root_field = 'score'
    l = len(df[df[f'team_cml_{cml_root_field}_after'] != (df[f'team_cml_{cml_root_field}_before']+df[f'team_{root_field}'])])
    print(f"[{l}] {root_field}")

[0] first_downs
[0] first_downs
[0] fumble_gained
[0] fumble_gained
[0] fumble_lost
[0] fumble_lost
[0] interceptions_gained
[0] interceptions_gained
[0] interceptions_lost
[0] interceptions_lost
[0] pass_completions
[0] pass_completions
[0] pass_count
[0] pass_count
[0] pass_yards
[0] pass_yards
[0] penalty_count
[0] penalty_count
[0] score
[0] score
[0] rush_count
[0] rush_count
[0] rush_yards
[0] rush_yards
[0] sack_count
[0] sack_count
[0] sack_gained
[0] sack_gained
[0] top_sec
[0] top_sec
[0] total_yards
[0] total_yards
[0] turnovers_gained
[0] turnovers_gained
[0] turnovers_lost
[0] turnovers_lost


# TESTER : CML Values for all data

The below shows that all fields are 50%, 55%, 45% and the like. Rerunning the below should get us data much more separated once things are fixed.

In [5]:
###
### CML PERCENTAGES
###


def get_perc(df, field):
    '''
    Given a df and a field, return the number of times the 
    winning team lead in that field.
    '''
    stat_leading_wins_df = df[
        # home team won      &  home team lead in stat
        ((df['win'] == 1) & (df[f'team_{field}'] > df[f'opponent_{field}'])) |
        # away team won      &  away team lead in stat
        ((df['win'] == 0) & (df[f'team_{field}'] <= df[f'opponent_{field}']))
      ]
    return (len(stat_leading_wins_df) / len(df)) * 100


def work_fields(df, fields):
    data = {}
    for field in fields:
        data[field] = get_perc(df, field)
    return data


games_df = dboss.games_df
# games_df = dboss.teams_df

# read in all CML columns and build an
# array of them (without team_ and opponent_)
cols = games_df.columns
team_cml_cols = pd.Series(cols[cols.str.contains('team_cml_')]).sort_values()
cml_cols = team_cml_cols.apply(lambda x: x.replace('team_', '')).values

data = work_fields(gu.get_year(games_df, 2020), cml_cols)

# array each item so it can be DF'd
for item in data:
    data[item] = [data[item]]
pdf = pd.DataFrame(data)

# pdf = pdf.T.sort_values(by=0).T
# pdf.T[0].sort_values(ascending=False)
# pdf.T
percs_df = pdf.T

percs_df.sort_values(by=0, ascending=False)


,0
cml_points_after,73.828125
cml_rush_count_after,64.453125
cml_rush_yards_after,62.890625
cml_turnovers_gained_after,62.890625
cml_top_sec_after,62.500000
cml_sack_gained_after,60.937500
cml_points_before,60.156250
cml_interceptions_gained_after,59.765625
cml_fumble_gained_after,58.593750
cml_total_yards_after,58.593750


In [6]:
percs_df.loc[[
'cml_pass_count_before','cml_pass_count_after',
'cml_pass_yards_before','cml_pass_yards_after',
'cml_rush_count_before','cml_rush_count_after',
'cml_rush_yards_before','cml_rush_yards_after',
]]

,0
cml_pass_count_before,47.656250
cml_pass_count_after,40.234375
cml_pass_yards_before,52.343750
cml_pass_yards_after,52.343750
cml_rush_count_before,54.296875
cml_rush_count_after,64.453125
cml_rush_yards_before,56.640625
cml_rush_yards_after,62.890625


In [55]:
before_fields = list(filter(lambda x : 'before' in x, list(percs_df.index)))
before_fields

['cml_comb_comp_perf_before',
 'cml_comb_def_perf_before',
 'cml_comb_off_perf_before',
 'cml_first_downs_before',
 'cml_fumble_gained_before',
 'cml_fumble_lost_before',
 'cml_interceptions_gained_before',
 'cml_interceptions_lost_before',
 'cml_pass_comp_perf_before',
 'cml_pass_completions_before',
 'cml_pass_count_before',
 'cml_pass_def_perf_before',
 'cml_pass_off_perf_before',
 'cml_pass_yards_before',
 'cml_penalty_count_before',
 'cml_points_before',
 'cml_rush_comp_perf_before',
 'cml_rush_count_before',
 'cml_rush_def_perf_before',
 'cml_rush_off_perf_before',
 'cml_rush_yards_before',
 'cml_sack_count_before',
 'cml_sack_gained_before',
 'cml_top_sec_before',
 'cml_total_yards_before',
 'cml_turnovers_gained_before',
 'cml_turnovers_lost_before']

In [63]:
percs_df.loc[before_fields].sort_values(by=0, ascending=False)


,0
cml_points_before,60.156250
cml_first_downs_before,56.640625
cml_rush_yards_before,56.640625
cml_turnovers_gained_before,55.859375
cml_total_yards_before,54.687500
cml_rush_count_before,54.296875
cml_sack_gained_before,52.734375
cml_pass_yards_before,52.343750
cml_top_sec_before,52.343750
cml_pass_completions_before,51.953125


In [7]:


def cmp_fields(field):
  return [
    'year_week',
    'team_score',
    'opponent_score',
    'win',
    f'team_{field}',
    f'opponent_{field}',
    ]

def check_df(df, field="pass_count"):
  # set up dfs for this field
  # home_wins_when_leading_df = df[((df['win'] == 1) & (df[f'team_cml_{field}_before'] > df[f'opponent_cml_{field}_before']))]
  # away_wins_when_leading_df = df[((df['win'] == 0) & (df[f'team_cml_{field}_before'] <= df[f'opponent_cml_{field}_before']))]
  any_wins_when_leading_df = df[(
      ((df['win'] == 1) & (df[f'team_cml_{field}_before'] > df[f'opponent_cml_{field}_before']))
      |
      ((df['win'] == 0) & (df[f'team_cml_{field}_before'] <= df[f'opponent_cml_{field}_before']))
    )]
  return round(len(any_wins_when_leading_df)/len(df) * 100,2)
  
for week in range(1,18):
  perc_winners = check_df(dboss.year_week(2020, week))
  print(f"{week}: {perc_winners}% winners") 
  
# print out the home wins
# home_wins_when_leading_df[cmp_fields(f'cml_{field}_before')]

1: 50.0% winners
2: 68.75% winners
3: 56.25% winners
4: 60.0% winners
5: 35.71% winners
6: 35.71% winners
7: 50.0% winners
8: 71.43% winners
9: 42.86% winners
10: 42.86% winners
11: 50.0% winners
12: 18.75% winners
13: 33.33% winners
14: 75.0% winners
15: 43.75% winners
16: 37.5% winners
17: 37.5% winners


## QUESTION

I am here wondering still about the balanced numbers above. It seems like what I need to do is spot-check a few like below. It looks like the `cml_pass_count_before` was calculated properly, and that the win by team or opponent is calculated properly... but when I spot check it with 1 week in a year there are few who fall into the "did win when" bucket. But maybe I'm not asking quite the right question.

In [26]:
def count_winners_while_leading_field(df, field):
  return len(df[(
  ((df['win'] == 1) & (df[f'team_{field}'] > df[f'opponent_{field}']))
  |
  ((df['win'] == 0) & (df[f'team_{field}'] <= df[f'opponent_{field}']))
)])
  
def get_perc(year, week=None, field="cml_pass_count_before" ):
  if week is None:
    df = dboss.year(year)
  else:
    df = dboss.year_week(year, week)
  
  win_count = count_winners_while_leading_field(df, field)
  return win_count / len(df) * 100

def get_final_perc(year, include_weeks=False, field="cml_pass_count_before"):
  if include_weeks is False:
    return get_perc(year, None, field)
  percs = []
  for week in range(1,18):
    perc = get_perc(year, week, field)
    percs.append(perc)
    print(f"{perc:.2f}% {year}:{week}")

  return np.array(percs).mean()

def work_years(start_year, end_year, fields, include_years=False, include_weeks=False):

  for field in fields:
    if include_years:
      print()
      print(field)
    
    percs = []
    for year in range(start_year, end_year + 1):
      perc = get_final_perc(year, include_weeks, field)
      percs.append(perc)
      if include_years:
        print(f"[{year}] {perc:.2f}%")
    
    overall = np.array(percs).mean()
    
    if include_years:
      print(f"       -------")
      print(f"       {overall:.2f}% overall")
    else:
      print(f"{overall:.2f}% {field}")
  
  
  
cml_before_fields = [k for k in cml_cols if 'before' in k]
work_years(2020, 2020, cml_before_fields, False, True)

50.00% 2020:1
31.25% 2020:2
62.50% 2020:3
60.00% 2020:4
35.71% 2020:5
50.00% 2020:6
57.14% 2020:7
50.00% 2020:8
64.29% 2020:9
21.43% 2020:10
35.71% 2020:11
62.50% 2020:12
66.67% 2020:13
56.25% 2020:14
50.00% 2020:15
43.75% 2020:16
56.25% 2020:17
50.20% cml_comb_comp_perf_before
50.00% 2020:1
31.25% 2020:2
62.50% 2020:3
60.00% 2020:4
35.71% 2020:5
50.00% 2020:6
57.14% 2020:7
50.00% 2020:8
64.29% 2020:9
21.43% 2020:10
35.71% 2020:11
62.50% 2020:12
66.67% 2020:13
56.25% 2020:14
50.00% 2020:15
43.75% 2020:16
56.25% 2020:17
50.20% cml_comb_def_perf_before
50.00% 2020:1
31.25% 2020:2
62.50% 2020:3
60.00% 2020:4
35.71% 2020:5
50.00% 2020:6
57.14% 2020:7
50.00% 2020:8
64.29% 2020:9
21.43% 2020:10
35.71% 2020:11
62.50% 2020:12
66.67% 2020:13
56.25% 2020:14
50.00% 2020:15
43.75% 2020:16
56.25% 2020:17
50.20% cml_comb_off_perf_before
50.00% 2020:1
56.25% 2020:2
68.75% 2020:3
73.33% 2020:4
50.00% 2020:5
21.43% 2020:6
42.86% 2020:7
57.14% 2020:8
64.29% 2020:9
64.29% 2020:10
64.29% 2020:11
43.75% 20

In [27]:
f1 = ['cml_rush_def_perf_before']
work_years(2020, 2020, f1, False, False)

50.39% cml_rush_def_perf_before


In [28]:
work_years(2020, 2020, f1, False, True)

50.00% 2020:1
31.25% 2020:2
62.50% 2020:3
60.00% 2020:4
35.71% 2020:5
50.00% 2020:6
57.14% 2020:7
50.00% 2020:8
64.29% 2020:9
21.43% 2020:10
35.71% 2020:11
62.50% 2020:12
66.67% 2020:13
56.25% 2020:14
50.00% 2020:15
43.75% 2020:16
56.25% 2020:17
50.20% cml_rush_def_perf_before


# THE ABOVE 2 SHOULD MATCH!

Turns out that doing 1 division for all the games in a season versus doing divisions for each week and then averaging those introduces this _rounding error_. It's small but real. But it is also unimportant for this investigation.

### By Hand : Total Year

In [41]:
field = 'cml_rush_def_perf_before'
year_df = dboss.year(2020)

count_all = len(year_df)
count_winners = count_winners_while_leading_field(year_df, field)
perc = count_winners / count_all * 100
print(f'{count_winners} / {count_all} count_winners / count_all')


print(f'{perc} perc')

129 / 256 count_winners / count_all
50.390625 perc


### By Hand : Weeks

In [43]:
percs = []
for week in range(1,18):
  df = dboss.year_week(2020, week)
  count_all = len(df)
  count_winners = count_winners_while_leading_field(df, field)
  perc = count_winners / count_all * 100
  percs.append(perc)

print(percs)
print(np.array(percs).mean())

[50.0, 31.25, 62.5, 60.0, 35.714285714285715, 50.0, 57.14285714285714, 50.0, 64.28571428571429, 21.428571428571427, 35.714285714285715, 62.5, 66.66666666666666, 56.25, 50.0, 43.75, 56.25]
50.203081232493
